In [ ]:
!pip install faiss-cpu # Install faiss library

In [ ]:
import faiss
import numpy as np
from sentence_transformers import SentenceTransformer
import json

# Sample metadata input (JSON format)

metadata_json ='/content/drive/MyDrive/Colab Notebooks/metadata.json'
with open(metadata_json, 'r') as file:
    metadata = json.load(file)
print(metadata)









{'table_descriptions': [{'table_name': 'actor', 'description': "The 'actor' table contains information about actors, including their unique actor IDs (aid), gender, name, nationality, city of birth, and year of birth. This table can be used to query demographic information about actors, group data by various attributes, and perform analytical operations such as ranking or comparisons.", 'columns': [{'name': 'actor.aid', 'description': 'A unique identifier for each actor. This field serves as the primary key of the table.'}, {'name': 'actor.gender', 'description': 'The gender of the actor, represented as a text field.'}, {'name': 'actor.name', 'description': 'The name of the actor, stored as text. This is the name by which the actor is known.'}, {'name': 'actor.nationality', 'description': 'The nationality of the actor, represented as text. This indicates the country the actor is associated with.'}, {'name': 'actor.birth_city', 'description': 'The city where the actor was born, stored a

In [ ]:
for table in metadata['table_descriptions']:
    if table['table_name'] == 'writer':
        table["description"]="The 'writer' table contains information about movie  writers, including their unique writer ID (wid), gender, name, nationality, city of birth, and birth year."

# Simple indexing

In [ ]:
# Load the model for sentence embeddings
model = SentenceTransformer('all-mpnet-base-v2')

NameError: name 'SentenceTransformer' is not defined

In [ ]:
# Function to create embeddings for text
def create_embeddings(texts):
    return model.encode(texts)

# Step 1: Index tables and columns using FAISS
def index_data(metadata):
    # Create lists for table and column descriptions
    table_descriptions = [table['description'] for table in metadata['table_descriptions']]
    column_descriptions = []
    column_refs = []

    for table in metadata['table_descriptions']:
        for column in table['columns']:
            column_descriptions.append(column["name"]+" " + column['description'])
            column_refs.append(column['name'])

    # Create embeddings for tables and columns
    table_embeddings = create_embeddings(table_descriptions)
    column_embeddings = create_embeddings(column_descriptions)
    print(column_descriptions)
    # FAISS index for tables
    table_index = faiss.IndexFlatL2(table_embeddings[0].shape[0])  # Using L2 distance (Euclidean distance)
    table_index.add(np.array(table_embeddings).astype('float32'))

    # FAISS index for columns
    column_index = faiss.IndexFlatL2(column_embeddings[0].shape[0])  # Using L2 distance (Euclidean distance)
    column_index.add(np.array(column_embeddings).astype('float32'))

    return table_index, column_index, column_refs, table_embeddings, column_embeddings


In [ ]:
# Step 2: Retrieve relevant tables and columns for a query
def retrieve_relevant_data(query, table_index, column_index, column_refs, table_embeddings, column_embeddings,metadata):
    # Embed the query using the same model
    query_embedding = create_embeddings([query])

    # Step 2.1: Retrieve relevant tables
    table_distances, table_indices = table_index.search(np.array(query_embedding).astype('float32'), k=4)  # Get top 4 relevant tables
    relevant_tables = [metadata['table_descriptions'][i]['table_name'] for i in table_indices[0]]

    # Step 2.2: Retrieve relevant columns for these tables
    relevant_columns = []
    for table in relevant_tables:
        # Filter columns related to the table
        table_columns = [column['description'] for column in next(t for t in metadata['table_descriptions'] if t['table_name'] == table)['columns']]

        column_distances, column_indices = column_index.search(np.array(query_embedding).astype('float32'), k=3)  # Get top 5 relevant columns
        for idx in column_indices[0]:
            relevant_columns.append(column_refs[idx])

    return relevant_tables, relevant_columns


In [ ]:
# Step 3: Index tables and columns
table_index, column_index, column_refs, table_embeddings, column_embeddings = index_data(metadata)




['actor.aid A unique identifier for each actor. This field serves as the primary key of the table.', 'actor.gender The gender of the actor, represented as a text field.', 'actor.name The name of the actor, stored as text. This is the name by which the actor is known.', 'actor.nationality The nationality of the actor, represented as text. This indicates the country the actor is associated with.', 'actor.birth_city The city where the actor was born, stored as text.', "actor.birth_year The year the actor was born, stored as an integer. This helps in determining the actor's age or generation.", 'cast.id A unique identifier for each entry in the cast table. This field serves as the primary key.', 'cast.msid A unique identifier for the production (movie, show, etc.) that the actor is associated with. It serves as a foreign key linking to other related tables.', 'cast.aid A unique identifier for the actor. This serves as a foreign key linking to an actor table containing detailed information 

In [ ]:
# print(table_index)
print(column_index)
# print(column_refs)
# print(table_embeddings)
# print(column_embeddings)

<faiss.swigfaiss_avx2.IndexFlatL2; proxy of <Swig Object of type 'faiss::IndexFlatL2 *' at 0x7c421daf7fc0> >


In [ ]:
# Example query
query = "Find all movies written and produced by 'Woody Allen'"

# Step 4: Retrieve relevant tables and columns for the query
relevant_tables, relevant_columns = retrieve_relevant_data(query, table_index, column_index, column_refs, table_embeddings, column_embeddings,metadata)

# Output
print("Relevant Tables:", relevant_tables)
print("Relevant Columns:", relevant_columns)

Relevant Tables: ['writer', 'producer', 'movie', 'genre']
Relevant Columns: ['directed_by.did', 'movie.release_year', 'movie.title', 'directed_by.did', 'movie.release_year', 'movie.title', 'directed_by.did', 'movie.release_year', 'movie.title', 'directed_by.did', 'movie.release_year', 'movie.title']


In [ ]:
from sklearn.metrics import precision_score, recall_score
import numpy as np

# Sample ground truth data
ground_truth_json ='/content/drive/MyDrive/Colab Notebooks/retrival_eval.json'
with open(ground_truth_json, 'r') as file:
    evaluation = json.load(file)

evaluation




{'queries': [{'query': "Find all movies produced by producer  'Walt Disney' after year 2010",
   'ground_truth': {'tables': ['COMPANY', 'COPYRIGHT', 'MOVIE'],
    'columns': ['COMPANY.NAME',
     'COPYRIGHT.CID',
     'COPYRIGHT.MSID',
     'MOVIE.TITLE',
     'MOVIE.RELEASE_YEAR']}},
  {'query': "What is the nationality of writer  'Kevin Spacey'?",
   'ground_truth': {'tables': ['WRITER'], 'columns': ['WRITER.NATIONALITY']}},
  {'query': "How much was the budget of  movie 'Finding Nemo'?",
   'ground_truth': {'tables': ['MOVIE'], 'columns': ['MOVIE.BUDGET']}},
  {'query': "Find all movies directed by director 'Steven Spielberg' after 2010",
   'ground_truth': {'tables': ['DIRECTED_BY', 'DIRECTOR', 'MOVIE'],
    'columns': ['DIRECTOR.NAME', 'MOVIE.TITLE', 'MOVIE.RELEASE_YEAR']}},
  {'query': "Who is the director of the movie 'James Bond'?",
   'ground_truth': {'tables': ['DIRECTED_BY', 'DIRECTOR', 'MOVIE'],
    'columns': ['DIRECTOR.NAME']}},
  {'query': "Find the actor who played role

In [ ]:
def calculate_precision_recall(ground_truth_columns, retrieved_columns):
    relevant_retrieved = len(set(ground_truth_columns) & set(retrieved_columns))
    precision = relevant_retrieved / len(retrieved_columns) if retrieved_columns else 0
    recall = relevant_retrieved / len(ground_truth_columns) if ground_truth_columns else 0
    if precision + recall > 0:
        f1 = 2 * (precision * recall) / (precision + recall)
    else:
        f1 = 0

    return precision, recall, f1



In [ ]:

def get_eval(retrieved_data,evaluation):
  queries = evaluation['queries']
  total_precision = 0
  total_recall = 0
  total_f1 = 0
  queries = evaluation['queries']
  num_queries = len(queries)
  import pandas as pd
  query_precisions = []
  query_recalls = []
  retrived_columns_list = []
  ground_truth_columns_list = []
  query_f1 =  []

  for i,query_data in enumerate(queries):
      ground_truth_columns = [column.lower()  for column in  query_data['ground_truth']['columns']]
      retrieved_columns = retrieved_data['retrieved_columns'][i]

      precision, recall, f1 = calculate_precision_recall(ground_truth_columns, retrieved_columns)
      query_precisions.append(precision)
      query_recalls.append(recall)
      query_f1.append(f1)
      total_precision += precision
      total_recall += recall
      total_f1 += f1
      retrived_columns_list.append([" ".join(retrieved_columns)])
      ground_truth_columns_list.append([" ".join(ground_truth_columns)])
  eval_df = pd.DataFrame({"queries":retrieved_data["query"],"ground_truth_cols":ground_truth_columns_list,"retrived_columns":retrived_columns_list,"query_precisions":query_precisions,"query_recalls":query_recalls,"query_f1":query_f1})
  eval_df.to_csv("retrival_eval.csv",index=False)


  # Calculate average precision and recall across all queries
  avg_precision = total_precision / num_queries
  avg_recall = total_recall / num_queries
  avg_f1 = total_f1 / num_queries
  # Output average precision and recall
  print(f"Average Precision: {avg_precision:.4f}")
  print(f"Average Recall: {avg_recall:.4f}")
  print(f"Average F1: {avg_f1:.4f}")



In [ ]:

retrieved_data = {"query":[],"retrieved_columns":[]}

queries = evaluation['queries']
from tqdm import tqdm
for query_data in tqdm(queries):
  query = query_data['query']
  relevant_tables, relevant_columns = retrieve_relevant_data(query, table_index, column_index, column_refs, table_embeddings, column_embeddings,metadata)
  retrieved_data["query"].append(query)
  retrieved_data["retrieved_columns"].append(relevant_columns)


get_eval(retrieved_data,evaluation)

100%|██████████| 21/21 [00:02<00:00,  8.18it/s]

Average Precision: 0.1111
Average Recall: 0.5063
Average F1: 0.1748


# heirarchical indexing

In [ ]:

# Function to create embeddings for text
def create_embeddings(texts):
    return model.encode(texts)
# Step 1: Index tables and columns using FAISS
def index_data(metadata):
    # Create lists for table and column descriptions
    table_descriptions = [table['description'] for table in metadata['table_descriptions']]
    column_descriptions = []
    column_refs = []
    column_indexes = {}

    # Index tables
    table_embeddings = create_embeddings(table_descriptions)
    table_index = faiss.IndexFlatL2(table_embeddings[0].shape[0])  # Using L2 distance (Euclidean distance)
    table_index.add(np.array(table_embeddings).astype('float32'))

    # Index columns for each table
    for table in metadata['table_descriptions']:
        table_columns = []
        table_column_refs = []
        for column in table['columns']:
            table_columns.append(column['description'])
            table_column_refs.append(column['name'])

        # Create embeddings for columns of the current table
        column_embeddings = create_embeddings(table_columns)

        # FAISS index for columns of this table
        column_index = faiss.IndexFlatL2(column_embeddings[0].shape[0])
        column_index.add(np.array(column_embeddings).astype('float32'))

        # Store column index and references
        column_indexes[table['table_name']] = (column_index, table_column_refs)

    return table_index, column_indexes, table_embeddings


In [ ]:
def retrieve_relevant_data(query, table_index, column_indexes, metadata):
    # Embed the query using the same model
    query_embedding = create_embeddings([query])

    # Step 2.1: Retrieve relevant tables
    table_distances, table_indices = table_index.search(np.array(query_embedding).astype('float32'), k=3)  # Get top 3 relevant tables
    relevant_tables = [metadata['table_descriptions'][i]['table_name'] for i in table_indices[0]]

    # Step 2.2: Retrieve relevant columns for these tables
    relevant_columns = []
    for table in relevant_tables:
        # Get column index and references for the relevant table
        column_index, column_refs = column_indexes[table]

        # Search for relevant columns from the current table
        column_distances, column_indices = column_index.search(np.array(query_embedding).astype('float32'), k=5)  # Get top 5 relevant columns

        # Collect relevant columns
        for idx in column_indices[0]:
            relevant_columns.append(column_refs[idx])

    return relevant_tables, relevant_columns

In [ ]:
# Step 3: Index tables and columns
table_index, column_indexes, table_embeddings = index_data(metadata)

NameError: name 'index_data' is not defined

In [ ]:
print(table_index)
print(column_indexes)

<faiss.swigfaiss_avx2.IndexFlatL2; proxy of <Swig Object of type 'faiss::IndexFlatL2 *' at 0x7c41c7f1a700> >
{'actor': (<faiss.swigfaiss_avx2.IndexFlatL2; proxy of <Swig Object of type 'faiss::IndexFlatL2 *' at 0x7c41c7f1bb70> >, ['actor.aid', 'actor.gender', 'actor.name', 'actor.nationality', 'actor.birth_city', 'actor.birth_year']), 'cast': (<faiss.swigfaiss_avx2.IndexFlatL2; proxy of <Swig Object of type 'faiss::IndexFlatL2 *' at 0x7c41c7f19560> >, ['cast.id', 'cast.msid', 'cast.aid', 'cast.role']), 'classification': (<faiss.swigfaiss_avx2.IndexFlatL2; proxy of <Swig Object of type 'faiss::IndexFlatL2 *' at 0x7c41c7f19dd0> >, ['classification.id', 'classification.msid', 'classification.gid']), 'company': (<faiss.swigfaiss_avx2.IndexFlatL2; proxy of <Swig Object of type 'faiss::IndexFlatL2 *' at 0x7c41c7f1a910> >, ['company.id', 'company.name', 'company.country_code']), 'copyright': (<faiss.swigfaiss_avx2.IndexFlatL2; proxy of <Swig Object of type 'faiss::IndexFlatL2 *' at 0x7c41b7b6

In [ ]:


# Example query
query = "Find all movies written and produced by 'Woody Allen'"

# Step 4: Retrieve relevant tables and columns for the query
relevant_tables, relevant_columns = retrieve_relevant_data(query, table_index, column_indexes, metadata)

# Output
print("Relevant Tables:", relevant_tables)
print("Relevant Columns:", relevant_columns)

Relevant Tables: ['writer', 'producer', 'movie']
Relevant Columns: ['writer.birth_year', 'writer.birth_city', 'writer.nationality', 'writer.gender', 'writer.name', 'producer.birth_city', 'producer.gender', 'producer.birth_year', 'producer.name', 'producer.nationality', 'movie.title', 'movie.release_year', 'movie.mid', 'movie.title_aka', 'movie.budget']


In [ ]:

retrieved_data = {"query":[],"retrieved_columns":[]}

queries = evaluation['queries']
from tqdm import tqdm
for query_data in tqdm(queries):
  query = query_data['query']
  relevant_tables, relevant_columns = retrieve_relevant_data(query, table_index, column_indexes, metadata)
  retrieved_data["query"].append(query)
  retrieved_data["retrieved_columns"].append(relevant_columns)


get_eval(retrieved_data,evaluation)

100%|██████████| 21/21 [00:03<00:00,  6.40it/s]

Average Precision: 0.1429
Average Recall: 0.7540
Average F1: 0.2335


In [ ]:
!pip install rank_bm25  # Install the missing 'rank_bm25' package

In [ ]:
import faiss
import numpy as np
from sentence_transformers import SentenceTransformer
from rank_bm25 import BM25Okapi
import json
from sklearn.preprocessing import MinMaxScaler





def index_data(metadata):
    table_descriptions = [table['description'] for table in metadata['table_descriptions']]
    column_refs = {}
    column_indices = {}

    for table in metadata['table_descriptions']:
        column_refs[table['table_name']] = []
        column_descriptions = []
        for column in table['columns']:
            column_refs[table['table_name']].append(column['name'])
            column_descriptions.append(column['description'])

        # Create FAISS index for columns
        column_embeddings = create_embeddings(column_descriptions)
        column_index = faiss.IndexFlatL2(column_embeddings[0].shape[0])
        column_index.add(np.array(column_embeddings).astype('float32'))
        column_indices[table['table_name']] = column_index

    # Vector-based embeddings for tables
    table_embeddings = create_embeddings(table_descriptions)

    # FAISS index for tables
    table_index = faiss.IndexFlatL2(table_embeddings[0].shape[0])
    table_index.add(np.array(table_embeddings).astype('float32'))

    # BM25 indices for tables
    table_bm25 = BM25Okapi([desc.split() for desc in table_descriptions])

    return table_index, table_bm25, column_indices, column_refs, table_embeddings

# RRF combination function
def rrf_combine(rankings, k=60):
    scores = {}
    for system_rankings in rankings:
        for rank, item in enumerate(system_rankings):
            scores[item] = scores.get(item, 0) + 1 / (k + rank + 1)
    return sorted(scores.items(), key=lambda x: x[1], reverse=True)

# Step 2: Perform hybrid retrieval
def hybrid_search(query, embeddings, bm25_index, faiss_index, top_k=3):
    # Vector-based search
    query_embedding = create_embeddings([query])
    if faiss_index:
        distances, indices = faiss_index.search(np.array(query_embedding).astype('float32'), k=top_k)
        faiss_ranking = indices[0].tolist()
    else:
        faiss_ranking = []

    # BM25 lexical search
    bm25_scores = bm25_index.get_scores(query.split())
    bm25_ranking = np.argsort(bm25_scores)[::-1][:top_k].tolist()

    # Combine rankings using RRF
    combined_ranking = rrf_combine([bm25_ranking, faiss_ranking])

    return [item[0] for item in combined_ranking[:top_k]]

# Step 3: Retrieve relevant data
def retrieve_relevant_data(query, table_index, table_bm25, column_indices, column_refs, metadata):
    # Retrieve relevant tables
    table_indices = hybrid_search(query, None, table_bm25, table_index)
    relevant_tables = [metadata['table_descriptions'][i]['table_name'] for i in table_indices]

    # Retrieve relevant columns
    relevant_columns = []
    for table in relevant_tables:
        bm25 = BM25Okapi([col.split() for col in column_refs[table]])
        faiss_index = column_indices[table]
        column_ranking = hybrid_search(query, None, bm25, faiss_index, top_k=5)
        relevant_columns.extend([column_refs[table][idx] for idx in column_ranking])

    return relevant_tables, relevant_columns

# Index data
table_index, table_bm25, column_bm25, column_refs, table_embeddings = index_data(metadata)

# Example query
query = "Find all movies written and produced by 'Woody Allen'"

# Retrieve relevant tables and columns for the query
relevant_tables, relevant_columns = retrieve_relevant_data(query, table_index, table_bm25, column_bm25, column_refs, metadata)

# Output results
print("Relevant Tables:", relevant_tables)
print("Relevant Columns:", relevant_columns)


Relevant Tables: ['movie', 'tags', 'writer']
Relevant Columns: ['movie.title', 'movie.release_year', 'movie.budget', 'movie.title_aka', 'movie.mid', 'tags.msid', 'tags.kid', 'tags.id', 'tags.kid', 'writer.birth_year', 'writer.birth_city', 'writer.nationality', 'writer.name', 'writer.gender']


In [ ]:
retrieved_data = {"query":[],"retrieved_columns":[]}

queries = evaluation['queries']
from tqdm import tqdm
for query_data in tqdm(queries):
  query = query_data['query']
  relevant_tables, relevant_columns = retrieve_relevant_data(query, table_index, table_bm25, column_bm25, column_refs, metadata)
  retrieved_data["query"].append(query)
  retrieved_data["retrieved_columns"].append(relevant_columns)


get_eval(retrieved_data,evaluation)

100%|██████████| 21/21 [00:11<00:00,  1.78it/s]

Average Precision: 0.1414
Average Recall: 0.6746
Average F1: 0.2260


In [ ]:
suggest_viz = '''
user_query : {user_query}
dataframe: {dataframe}

Visualisation Options: ['time series', 'bar plot', 'map', 'scatter_plot', 'piechart', 'None'].List of Visualization options.

Goal: Suggest suitable visualization(s) type from the following options to effectively represent the data for the given user question and the generated dataframe.

Guidelines:

Your recommendation should be based on only the given options.

If there is no suitable visualization return empty list.

The output should be formatted as a JSON instance that conforms to the 350N schema below.

As an example, for the valid  schema ("properties": {"foo": ("title": "foo", "description": "a list of strings", "type": "array", "items": ("type": "string"))}, "required": ["foo"]} the object ["foo": ["bar", "baz"]) is a well-formatted instance of the schema.
The object ("properties": ("foo": ["bar", "baz"]}) is not well-formatted.

Here is the output schema:

("properties": {"visualization type": ("description": "Suggested visualization type(s) for the given user question and the generated dataframe", "items": ["type": "string"), "title": "Visualization Type", "type": "array"}}, "required": ["visualization_type]}'''

In [ ]:
suggest_viz.format(user_query=query,dataframe=metadata)